# <center> <span style="color:#f6f794"> 🧼 CLEANING DATA </span> </center>

_____

### <span style="color:#f6f794"> 📚 Importing libraries and setting displays </span>

In [ ]:
import pandas as pd
import numpy as np
import requests
import gzip
from io import BytesIO
import os
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import time
import csv
import re


### <span style="color:#f6f794"> 📂 Opening files and create dataframes of them </span>

In [3]:
# Setting paths
current_dir = os.getcwd()                            # \sentiment-analyzer-inc\eda
print(current_dir)
parent_dir = os.path.dirname(current_dir)            # \sentiment-analyzer-inc
print(parent_dir)
data_dir = os.path.join(current_dir, "data")          # \sentiment-analyzer-inc\eda\data
print(data_dir)
cleaning_file_path = os.path.join(current_dir, "")      # \sentiment-analyzer-inc\eda\data\
print(cleaning_file_path)

c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\data
c:\Users\yaelp\Desktop\Yael\Bootcamp_FactoriaF5_IA_Promo4\Proyectos\Grupales\sentiment-analyzer-inc\eda\


In [13]:


GITHUB_CLEAN_URL = "https://raw.githubusercontent.com/Yael-Parra/sentiment-analyzer-inc/feature/data_youtube/etl/data/youtube_comments_-4GmbBoYQjE.csv.gz"

def load_airbnb_data_from_github(url):
    """
    Downloading and processing Airbnb data from GitHub repository.
    """
    print("⏯️ Downloading data from GitHub...")

    try:
        # Download the compressed CSV file from GitHub
        response = requests.get(url)
        response.raise_for_status()

        print("⏯️ bar_chart: Processing compressed file..")

        # Decompressing and reading the CSV file
        with gzip.open(BytesIO(response.content), 'rt', encoding='utf-8') as f:
            df = pd.read_csv(
                f,
                sep=',',                  # Specify the delimiter
                encoding='utf-8',         # Specify the encoding
                quoting=csv.QUOTE_MINIMAL # Specify the quoting behavior
                )

        print(f"✅ white_check_mark: Data downloaded succesfully!")
        
        return df

    except requests.exceptions.RequestException as req_err:
        print(f"⚠️ Network/Request Error while downloading data: {req_err}")
        return None
    except Exception as e:
        print(f"⚠️ General Error while processing data: {e}")
        return None

# Creating dataframe from GitHub URL
df_youtube_extract = load_airbnb_data_from_github(GITHUB_CLEAN_URL)

⏯️ Downloading data from GitHub...
⏯️ bar_chart: Processing compressed file..
✅ white_check_mark: Data downloaded succesfully!


___________
# <center> <span style="color:#f6f794">  🧹🧹🧹🧹🧹🧹🧹 START CLEANING 🧹🧹🧹🧹🧹🧹🧹 </center>
__________

In [14]:
df_youtube_extract.columns

Index(['threadId', 'videoId', 'commentId', 'author', 'publishedAt',
       'updatedAt', 'text', 'likeCount', 'replyCount', 'canReply', 'isPublic',
       'parentId'],
      dtype='object')

In [15]:
df_youtube_extract

,threadId,videoId,commentId,author,publishedAt,updatedAt,text,likeCount,replyCount,canReply,isPublic,parentId
0,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,Ugy0E7NVmoLzOdofHcF4AaABAg,@MrBeast,2025-05-10T16:17:59Z,2025-05-10T17:54:04Z,"SUBSCRIBE TO WIN $50,000! To celebrate my birt...",65097,638.0,False,True,NaN
1,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBmoK2NNH,@2005hondaaccord-i4x,2025-05-10T16:18:14Z,2025-05-10T16:18:14Z,First,778,NaN,NaN,NaN,Ugy0E7NVmoLzOdofHcF4AaABAg
2,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBp6bpcSn,@zkgamer0075,2025-05-10T16:18:33Z,2025-05-10T16:18:33Z,Ok,341,NaN,NaN,NaN,Ugy0E7NVmoLzOdofHcF4AaABAg
3,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBpEdvOSs,@حيدر-حيدر313,2025-05-10T16:18:34Z,2025-05-10T16:18:34Z,قبل 28 ثانيه,246,NaN,NaN,NaN,Ugy0E7NVmoLzOdofHcF4AaABAg
4,Ugy0E7NVmoLzOdofHcF4AaABAg,-4GmbBoYQjE,Ugy0E7NVmoLzOdofHcF4AaABAg.AHxBl1q6T3rAHxBpdWOXYa,@Disaster33,2025-05-10T16:18:37Z,2025-05-10T16:18:37Z,Wow😮,155,NaN,NaN,NaN,Ugy0E7NVmoLzOdofHcF4AaABAg
...,...,...,...,...,...,...,...,...,...,...,...,...
6996,Ugy60PH_kYKCZTm0vyJ4AaABAg,-4GmbBoYQjE,Ugy60PH_kYKCZTm0vyJ4AaABAg,@peguerojordy,2025-05-23T03:30:48Z,2025-05-23T03:30:48Z,Thank you for supporting communities and doing...,8,0.0,True,True,NaN
6997,UgxktbSIynNNhL7Jsvd4AaABAg,-4GmbBoYQjE,UgxktbSIynNNhL7Jsvd4AaABAg,@SarkiriEngti-sy8oh,2025-05-23T03:30:47Z,2025-05-23T03:30:47Z,Love you Mrbeats❤,1,0.0,True,True,NaN
6998,UgxJlTdyGF2e3cmAnfJ4AaABAg,-4GmbBoYQjE,UgxJlTdyGF2e3cmAnfJ4AaABAg,@reyanshgodha2966,2025-05-23T03:29:11Z,2025-05-23T03:29:11Z,I subbed,0,0.0,True,True,NaN
6999,Ugx_uww7gCceEX-C8cR4AaABAg,-4GmbBoYQjE,Ugx_uww7gCceEX-C8cR4AaABAg,@BIJOYSAHA-b9x,2025-05-23T03:27:48Z,2025-05-23T03:27:48Z,আপনি দিনে কত টাকা 💰💸 আয় করেন আমি আপনার আরো ভি...,1,0.0,True,True,NaN


In [16]:
df_youtube_extract.shape # df_clean_2.shape
df_youtube_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7001 entries, 0 to 7000
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   threadId     7001 non-null   object 
 1   videoId      7001 non-null   object 
 2   commentId    7001 non-null   object 
 3   author       6997 non-null   object 
 4   publishedAt  7001 non-null   object 
 5   updatedAt    7001 non-null   object 
 6   text         6993 non-null   object 
 7   likeCount    7001 non-null   int64  
 8   replyCount   6839 non-null   float64
 9   canReply     6839 non-null   object 
 10  isPublic     6839 non-null   object 
 11  parentId     162 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 656.5+ KB


#### <span style="color:#F3A006"> 1️⃣ Columns </span>

- Column's names format: lowercase with underscore as spaces

In [18]:
# Changin the names to lowercase and with a underscore to as spaces to make it easier to work with.
df_youtube_extract.columns = (
    df_youtube_extract.columns
    .str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True)  # Insert underscore before each uppercase (except start)
    .str.lower()                                       # Convert all to lowercase
    .str.strip()                                       # Remove leading and trailing spaces in case there were any
)

In [19]:
df_youtube_extract.columns

Index(['thread_id', 'video_id', 'comment_id', 'author', 'published_at',
       'updated_at', 'text', 'like_count', 'reply_count', 'can_reply',
       'is_public', 'parent_id'],
      dtype='object')